In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [3]:
# 864691128455135308
edges = np.array([[94879882017803318, 94809513273628034]])
print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_roots(edges[0]))

[4]
[864691138381365262 864691139356549869]


In [7]:
# parents = np.uint64(edges[0][0])
# while parents and parents.size:
#     parents = cg.get_parent(parents)
#     print(parents)

In [6]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edges(cg, atomic_edges=edges))

Start add_edge
   Start get_parents edges
   1.8596649169921875e-05
Cannot find root id 382986288233447425, 8, 2020-02-10 19:08:10.331000+00:00
Cannot find root id 382986288233447426, 8, 2020-02-10 19:08:11.247000+00:00
Cannot find root id 600386125323829254, 12, 2020-02-10 19:08:13.631000+00:00
Cannot find root id 600386125323829255, 12, 2020-02-10 19:08:14.235000+00:00
   [864691128455135305, 864691128455135306]
9.874366521835327
